## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Rishabh Sharma
    - Email: rs3738@drexel.edu
- Group member 2
    - Name: Shai Wudkwych
    - Email: sw3468@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment group 2: Network and exploratory data analysis

## Module D _(40 pts)_ An ingredient-based recommender system
In this module we're going to build a recommender system using some recipes data and the Apriori algorithm. These data can be obtained from Kaggle:

- https://www.kaggle.com/kaggle/recipe-ingredients-dataset

and are packaged with the assignment in the following directory:

- `./data/train.json`

__D1.__ _(2 pts)_ To start, write a function called `read_recipes`, which takes a string argument called `path_to_recipes_json` that contains the path to a json file containing recipe data. The function should use the `json` package to load the data and then return `recipes`, which will be a list of dictionaries containing the converted json data.

(_Hint_: This function will be identical to the one you wrote for _C1_.)

In [29]:
# D1:Function(2/2)

import json

def read_recipes(path_to_recipes_json):
    #--- Your code starts here
    with open(path_to_recipes_json) as f:
        recipes = json.load(f)
    #--- Your code ends here
    
    return recipes


To test your function, let's provide it with the path to the `train.json` data and print the first three recipes.

Your output should look like this:

```
{'id': 10259, 'cuisine': 'greek', 'ingredients': ['feta cheese crumbles', 'garlic', 'seasoning', 'grape tomatoes', 'black olives', 'garbanzo beans', 'pepper', 'purple onion', 'romaine lettuce']} 

{'id': 25693, 'cuisine': 'southern_us', 'ingredients': ['ground pepper', 'ground black pepper', 'vegetable oil', 'plain flour', 'thyme', 'salt', 'green tomatoes', 'milk', 'yellow corn meal', 'eggs', 'tomatoes']} 

{'id': 20130, 'cuisine': 'filipino', 'ingredients': ['butter', 'green chilies', 'cooking oil', 'chicken livers', 'pepper', 'salt', 'grilled chicken breasts', 'garlic powder', 'soy sauce', 'mayonaise', 'yellow onion', 'eggs']}
```

In [72]:
# D1:SanityCheck

recipes = read_recipes('./data/train.json')

for recipe in recipes[:3]:
    print(recipe,"\n")

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['feta cheese crumbles', 'garlic', 'seasoning', 'grape tomatoes', 'black olives', 'garbanzo beans', 'pepper', 'purple onion', 'romaine lettuce']} 

{'id': 25693, 'cuisine': 'southern_us', 'ingredients': ['ground pepper', 'ground black pepper', 'vegetable oil', 'plain flour', 'thyme', 'salt', 'green tomatoes', 'milk', 'yellow corn meal', 'eggs', 'tomatoes']} 

{'id': 20130, 'cuisine': 'filipino', 'ingredients': ['butter', 'green chilies', 'cooking oil', 'chicken livers', 'pepper', 'salt', 'grilled chicken breasts', 'garlic powder', 'soy sauce', 'mayonaise', 'yellow onion', 'eggs']} 



__D2.__ _(5 pts)_ Next, write a function called `count_items` that takes the recipes data you loaded in _D1_ (`recipes`) and uses a `Counter` (i.e., the `counts` object) to count up the number of recipes that include each `ingredient`, storing each in the counter as a single-element tuple (for downstream convenience), i.e., incrementing like `counts[tuple([ingredient])] +=1`. The function returns the populated `counts` object.

In [73]:
# D2:Function(5/5)

from collections import Counter

def count_items(recipes):
    #---Your code starts here
    counts = Counter()
    for rec in recipes:
        for item in rec['ingredients']:
            counts[tuple([item])] += 1 
    #---Your code ends here
    
    return counts

To test your `count_items` function, let's apply it to the `recipes` loaded in _D1_ and then print the count for "salt" as well as the total number of candidates (i.e., ingredients) in the output. The output should be:
```
Count for salt: 18048
Total # of candidates: 6714
```

In [74]:
# D2:SanityCheck

candidates_one = count_items(recipes)

print("Count for salt: {}".format(candidates_one[('salt',)]))
print("Total # of candidates: {}".format(len(candidates_one)))

Count for salt: 18048
Total # of candidates: 6714


__D3.__ _(5 pts)_ Now, write a function called `store_frequent(candidates, threshold = 25)`, which accepts a `Counter` of `candidates`, i.e., item or itemset counts, and stores only those with count above the determined `threshold` value in a separate counter called `frequent`, which is `return`ed at the end of the function.

In [75]:
# D3:Function(5/5)

def store_frequent(candidates, threshold = 25):
    #---Your code starts here
    frequent = Counter()
    for i in candidates:
        if candidates[i] >= threshold:
            frequent[i] = candidates[i]
    #---Your code ends here
            
    return frequent

To test this function, let's apply it to the `candidates` (output from `count_items(...)`) we generated in _D2_ with a threshold of `4000` and look at the candidates that are above this threshold. The output should be:
```
Counter({('garlic',): 7380,
         ('pepper',): 4438,
         ('ground black pepper',): 4784,
         ('vegetable oil',): 4385,
         ('salt',): 18048,
         ('butter',): 4847,
         ('water',): 7457,
         ('onions',): 7972,
         ('sugar',): 6434,
         ('olive oil',): 7971,
         ('garlic cloves',): 6236,
         ('all-purpose flour',): 4632})
```

In [76]:
# D3:SanityCheck
frequent = store_frequent(candidates_one, 4000)
frequent

Counter({('garlic',): 7380,
         ('pepper',): 4438,
         ('ground black pepper',): 4784,
         ('vegetable oil',): 4385,
         ('salt',): 18048,
         ('butter',): 4847,
         ('water',): 7457,
         ('onions',): 7972,
         ('sugar',): 6434,
         ('olive oil',): 7971,
         ('garlic cloves',): 6236,
         ('all-purpose flour',): 4632})

__D4.__ (10 pts) Now, write a function called `get_next(recipes, frequent, threshold = 25)` that accepts the `recipies` from _D1_, a `frequent` object (output from the `store_frequent()` function), and a `threshold`. With these inputs, your function should:

1. create a new `Counter` called `next_candidates`
2. compute the `size` of the itemsets for `next_candidates` from a single key in `frequent` (e.g., if an element in frequent is `('salt',)` then the size will be 2 and if an element in frequent is `('onions', 'salt')`, then the size is 3).
3. `for` any `recipe` with _at least_ as many ingredients as `size`:
    1. loop over all itemsets of size `size` (see combinations note below)
    2. utilize the apriori principle (see __section 4.2.2.6__) and subsets of itemsets to count up potentially-frequent candidate itemsets in `next_candidates`
4. `return(next_candidates)`

Note: To complete this part it is _extremely strongly_ encouraged that you import the `combinations()` function from the `itertools` module. With this, you can execute `combinations(items, k)` to find all combinations of size `k` from a list of `items`.


In [47]:
# D4:Function(8/10)

from itertools import combinations

def get_next(recipes, frequent, threshold = 25):
    
    #---Your code starts here
    next_candidates = Counter()
    size = len(list(frequent.keys())[0]) + 1
    for recipe in recipes:
        
        if len(recipe['ingredients']) >= size:
            for itemset in combinations(recipe['ingredients'], size):
                itemset = tuple(sorted(itemset))
                if itemset in next_candidates:
                    next_candidates[itemset] += 1
                elif all([tuple(sorted(subset)) in frequent 
                          for subset in combinations(itemset, size - 1)]):
                    next_candidates[itemset] += 1
    #---Your code ends here    
    return next_candidates

To test our function, lets first create a `frequent_one` list that contains terms that occur at least 25 times in our `candidates_one` list. Then we'll create the `candidates_two` list using our new `get_next` function and a `frequent_two` list that contains the terms from this new candidate list that occur at least 25 times. We'll print the lengths of `candidates_two` and `frequent_two` as well as the 10 most common words on the `frequent_two` list. Your output should look like this:
```
Length of candidates_two: 283161
Length of frequent_two: 15230

10 most common from frequent_two:
[(('onions', 'salt'), 4392),
 (('olive oil', 'salt'), 4177),
 (('salt', 'water'), 3960),
 (('pepper', 'salt'), 3844),
 (('garlic', 'salt'), 3749),
 (('all-purpose flour', 'salt'), 3079),
 (('salt', 'sugar'), 3061),
 (('garlic cloves', 'salt'), 2995),
 (('butter', 'salt'), 2777),
 (('ground black pepper', 'salt'), 2734)]
```

In [48]:
# D4:SanityCheck

from pprint import pprint

frequent_one = store_frequent(candidates_one)
candidates_two = get_next(recipes, frequent_one)
frequent_two = store_frequent(candidates_two)

print("Length of candidates_two: {}".format(len(candidates_two)))
print("Length of frequent_two: {}".format(len(frequent_two)))
print()
print("10 most common from frequent_two:")
pprint(frequent_two.most_common(10))

Length of candidates_two: 283161
Length of frequent_two: 15230

10 most common from frequent_two:
[(('onions', 'salt'), 4392),
 (('olive oil', 'salt'), 4177),
 (('salt', 'water'), 3960),
 (('pepper', 'salt'), 3844),
 (('garlic', 'salt'), 3749),
 (('all-purpose flour', 'salt'), 3079),
 (('salt', 'sugar'), 3061),
 (('garlic cloves', 'salt'), 2995),
 (('butter', 'salt'), 2777),
 (('ground black pepper', 'salt'), 2734)]


Next, to further test our function, lets apply our `get_next` function to the `frequent_two` list to build up a `candidate_three` list. We will then apply `store_frequent` to this new list to build up a `frequent_three` list. Like before, we'll print the lengths of `candidates_three` and `frequent_three` as well as the 10 most common words on the `frequent_three` list. Your output should look like this:
```
Length of candidates_three: 212328
Length of frequent_three: 24289

10 most common from frequent_three:
[(('garlic', 'onions', 'salt'), 1605),
 (('onions', 'pepper', 'salt'), 1342),
 (('onions', 'salt', 'water'), 1240),
 (('olive oil', 'onions', 'salt'), 1203),
 (('garlic', 'olive oil', 'salt'), 1185),
 (('garlic', 'pepper', 'salt'), 1170),
 (('olive oil', 'pepper', 'salt'), 1164),
 (('garlic cloves', 'olive oil', 'salt'), 1130),
 (('all-purpose flour', 'salt', 'sugar'), 954),
 (('ground black pepper', 'olive oil', 'salt'), 953)]
```

In [49]:
# D4:SanityCheck

candidates_three = get_next(recipes, frequent_two)
frequent_three = store_frequent(candidates_three)

print("Length of candidates_three: {}".format(len(candidates_three)))
print("Length of frequent_three: {}".format(len(frequent_three)))
print()
print("10 most common from frequent_three:")
pprint(frequent_three.most_common(10))

Length of candidates_three: 212328
Length of frequent_three: 24289

10 most common from frequent_three:
[(('garlic', 'onions', 'salt'), 1605),
 (('onions', 'pepper', 'salt'), 1342),
 (('onions', 'salt', 'water'), 1240),
 (('olive oil', 'onions', 'salt'), 1203),
 (('garlic', 'olive oil', 'salt'), 1185),
 (('garlic', 'pepper', 'salt'), 1170),
 (('olive oil', 'pepper', 'salt'), 1164),
 (('garlic cloves', 'olive oil', 'salt'), 1130),
 (('all-purpose flour', 'salt', 'sugar'), 954),
 (('ground black pepper', 'olive oil', 'salt'), 953)]


In [50]:
# D4:Inline

# Does running the above processes become more computationally
# expensive as the size gets larger? Print "Yes" or "No"
print("Yes")

Yes


__D5.__ (10 pts) Now that we have the pieces to run Apriori/collect frequent itemsets it's time to package the process together, collecting all frequent itemsets up to a particular `size`. To do this, write a function called `train(recipes, size = 4)`, which:

1. initializes two empty dictionaries, `candidates`, and `frequent`;
2. runs the `count_items` and `store_frequent` function, storing output in the `candidates`, and `frequent` dictionaries using the integer `1` as a key;
3. loops over sizes: 2, 3, .., `size` to compute and store the subsequent sizes candidates and frequent itemsets in the same structure as (2), but now utilizing the `get_next` function, instead of `count_items`; and
4. `return`s the `candidates` and `frequent` itemsets.

In [77]:
def train(recipes, size = 4):
    
    #---Your code starts here
    candidates, frequent = dict(), dict()
    candidates[1] = count_items(recipes)
    frequent[1] = store_frequent(candidates[1])
    for i in range(2, size+1):
        candidates[i] = get_next(recipes, frequent[i-1])
        frequent[i] = store_frequent(candidates[i])
    #---Your code ends here
    
    return candidates, frequent

To test your `train` function, let's apply it to the recipes data and build up the candidate and frequent lists up to size 4. Let's then print out the size of the candidates and frequent for different sizes. Your output should look like this:
```
size=1, # candidates=6714, # frequent=1486
size=2, # candidates=283161, # frequent=15230
size=3, # candidates=212328, # frequent=24289
size=4, # candidates=45194, # frequent=12249
```

In [78]:
size = 4
candidates, frequent = train(recipes, size)
for i in range(1,size + 1):
   print("size={}, # candidates={}, # frequent={}".format(i, len(candidates[i]), len(frequent[i])))

size=1, # candidates=6714, # frequent=1486
size=2, # candidates=283161, # frequent=15230
size=3, # candidates=212328, # frequent=24289
size=4, # candidates=45194, # frequent=12249


__D6.__ _(8 pts)_ Now that we have our `frequent` itemsets up to `size`, we can utilize them to recommend missing ingredients from ingredient 'baskets' of at most `size - 1`. To do this, write a function called `recommend(basket, frequent)` that does the following: 

1. initializes an empty `recommendations` list
2. loops over all frequent `itemset`s of `size 1 greater than the `basket`
    - if there's one item left from the `itemset` when the `basket` elements have been removed, append the remaining item to the `recommendations` list in a tuple, with the number of ocurrences of the itemset in the second position
4. `return` `recommendations`, but sorted from high to low by itemset ocurrence.

Additionally: Your function should return the string `'No candidates computed for baskets of this size!'` when appropriate to alert the user to not having trained on itemsets large enough.

In [67]:
# D6:Function(7/8)

def recommend(basket, frequent):
    #---Your code starts here
    basket = set(basket)
    if len(basket)+1 not in frequent:
        recommendations = 'No Candidates Computed for Baskets of this Size!'
    else:
        recommendations = []
        for items in frequent[len(basket)+1]:
            remaining = set(items) - basket
            if len(remaining) == 1:
                recommendations.append((remaining.pop(), frequent[len(basket)+1][items]))
        recommendations = sorted(recommendations, key = lambda x: x[1], reverse = True)
    #---Your code ends here 
    
    return recommendations

To test your `recommend` function, let's pass in the `frequent` object we generated with our `train` function and then use it to identify the top 10 recommendations for the following: `basket = tuple(['butter', 'flour'])`. Your output should look like this:
```
[('salt', 306),
 ('milk', 155),
 ('sugar', 149),
 ('eggs', 137),
 ('onions', 104),
 ('pepper', 103),
 ('baking powder', 81),
 ('garlic', 74),
 ('water', 70),
 ('olive oil', 52)]
```

In [68]:
# D6:SanityCheck
recommend(tuple(['butter', 'flour']), frequent)[:10]

[('salt', 306),
 ('milk', 155),
 ('sugar', 149),
 ('eggs', 137),
 ('onions', 104),
 ('pepper', 103),
 ('baking powder', 81),
 ('garlic', 74),
 ('water', 70),
 ('olive oil', 52)]

Let's test your function one more time by using it to identify the top 10 recommendations for the following: `basket = tuple(['avocado', 'garlic', 'salt'])`. Your output should look like this:
```
[('olive oil', 61),
 ('lime', 61),
 ('pepper', 52),
 ('cilantro', 48),
 ('onions', 44),
 ('chili powder', 43),
 ('cumin', 43),
 ('jalapeno chilies', 39),
 ('sour cream', 38),
 ('ground cumin', 35)]
```

In [69]:
# D6:SanityCheck
recommend(tuple(['avocado', 'garlic', 'salt']), frequent)[:10]

[('olive oil', 61),
 ('lime', 61),
 ('pepper', 52),
 ('cilantro', 48),
 ('onions', 44),
 ('chili powder', 43),
 ('cumin', 43),
 ('jalapeno chilies', 39),
 ('sour cream', 38),
 ('ground cumin', 35)]

In [71]:
# D6:Inline(1/8)

# Does the output of our recommender seem appropriate?
# Print "Yes" or "No"
print("Yes")

Yes
